In [ ]:
import datetime
from ray import train, tune, air
from ray.rllib.algorithms.sac import SACConfig
from common.envUtils import *

TASK="Reach_"
experiment_name = TASK + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
LOGDIR=f"/home/ey/rl/src/rlreach2/rlreach/ray/db/ray_results/{experiment_name}"

config = (
    SACConfig()
    .environment(
        env=ReachEnvGym,
        env_config={"log_dir": LOGDIR},        
    )
    .training(
        initial_alpha=0.2,
        actor_lr=1e-4,
        critic_lr=1e-4,
        alpha_lr=1e-4,
        target_entropy="auto",
        n_step=1,
        tau=0.005,
        train_batch_size=128,
        target_network_update_freq=1,
        replay_buffer_config={
            "type": "EpisodeReplayBuffer",
            "capacity": 1000000,
            "learning_starts": 1000,
            # HER 专用参数
            "replay_mode": "independent",
            "replay_sequence_length": 1,
            "replay_burn_in": 0,
            "replay_zero_init_states": False,
            "storage_unit": "episodes",
            # 关键：HER wrapper 配置
            "wrap_buffer": True,
            "wrapped_buffer": {
                "type": "HindsightExperienceReplayBuffer",
                "replay_mode": "independent",
                "her_strategy": "future",      # 可选: future, final, episode
                "replay_k": 4,                 # 每个 transition 生成多少个 HER 样本
                "goal_fn": None,               # 你可以自定义 goal extraction function
            },
        },
        num_steps_sampled_before_learning_starts=1000,
        model={
            "fcnet_hiddens": [512, 512],
            "fcnet_activation": "relu",
            "post_fcnet_hiddens": [],
            "post_fcnet_activation": None,
            "post_fcnet_weights_initializer": "orthogonal_",
            "post_fcnet_weights_initializer_config": {"gain": 0.01},
        },
    )
    .resources(
        num_gpus=0.25,      # 或 0.25 视机器配置
        num_cpus_per_worker=1,
        num_learner_workers=1,
    )
    .framework("torch")
    .reporting(
        metrics_num_episodes_for_smoothing=5,
        min_sample_timesteps_per_iteration=1000,
    )
    .evaluation(
        evaluation_interval=1,
        evaluation_num_env_runners=1,
        evaluation_config={"seed": 42},
    )
    .env_runners(
        num_env_runners=5,             # 进程数量
        num_envs_per_env_runner=1,     # 环境数量
        # gym_env_vectorize_mode="ASYNC"
    )
)


tunner = tune.Tuner(
    trainable=config.algo_class,
    param_space=config,
    run_config=train.RunConfig(
        name="reach",
        storage_path=LOGDIR,
        log_to_file=True,
        checkpoint_config=air.CheckpointConfig(
            checkpoint_frequency=10,
            checkpoint_at_end=True,
        ),
        stop={"evaluation/env_runners/episode_return_mean": 12000.0}
    ),
)

results = tunner.fit()

2025-08-13 14:00:31,604	WARNING algorithm_config.py:5033 -- You are running SAC on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
2025-08-13 14:00:31,605	WARNING algorithm_config.py:5062 -- You configured a custom `model` config (probably through calling config.training(model=..), whereas your config uses the new API stack! In order to switch off the new API stack, set in your config: `config.api_stack(enable_rl_module_and_learner=False, enable_env_runner_and_connector_v2=False)`. If you DO want to use the new API stack, configure your model, instead, through: `config.rl_module(model_config={..})`.
2025-08-13 14:00:31,606	WARNING sac.py:487 -- You are running SAC on the new API stack! This is the new default

(SingleAgentEnvRunner pid=6891) [chatbus_1] 共享内存不存在，创建成功
(SingleAgentEnvRunner pid=6895) [chatbus_1] 共享内存已存在，连接成功


(SingleAgentEnvRunner pid=6891) /home/ey/anaconda3/envs/rlreach310/lib/python3.10/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
(SingleAgentEnvRunner pid=6891)   gym.logger.warn(
(SingleAgentEnvRunner pid=6891) /home/ey/anaconda3/envs/rlreach310/lib/python3.10/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
(SingleAgentEnvRunner pid=6891)   gym.logger.warn(
(SingleAgentEnvRunner pid=6891) 2025-08-13 14:00:44,504	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!
(SAC pid=6776) 2025-08-13 14:00:44,673	WARNING algorithm_config.py:5062 -- You configured a custom `model` config (probably thro

(SingleAgentEnvRunner pid=7302) [chatbus_3] 共享内存不存在，创建成功 [repeated 2x across cluster]
(SingleAgentEnvRunner pid=6901) [chatbus_1] 共享内存已存在，连接成功 [repeated 3x across cluster]


(pid=7372) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=7372) E0000 00:00:1755064851.882582    7372 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=7372) E0000 00:00:1755064851.886191    7372 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=7372) W0000 00:00:1755064851.895047    7372 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once. [repeated 4x across cluster]
(_WrappedExecutable pid=7372) [robosuite WARNING] No private macro file found! (macros.py:57)
(_WrappedExecutable pid=7372) [robosuite WARNING] It is recommended to use a private macro file (macros.py:58)
(_WrappedExecutable pid=7372) [robosuite WARNING] To setup, run: python /home/ey/rl/src/robosuite/robosuite

In [ ]:
from pathlib import Path
import gymnasium as gym
import numpy as np 
import torch
from ray.rllib.core.rl_module import RLModule
from ray.rllib.algorithms.sac import SACConfig
from ray.rllib.models.torch.torch_distributions import TorchDiagGaussian
from common.envUtils import *

TASK="Reach_"
experiment_name = TASK + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
LOGDIR=f"/home/ey/rl/src/rlreach2/rlreach/ray/db/ray_results/{experiment_name}"

checkpoint_path = "/home/ey/rl/src/rlreach2/rlreach/ray/db/ray_results/Reach_2025-08-09_16-32-52/reach/SAC_ReachEnvGym_714b7_00000_0_2025-08-09_16-32-53/checkpoint_000283"
rl_module = RLModule.from_checkpoint(
    Path(checkpoint_path)
    / "learner_group"
    / "learner"
    / "rl_module"
    / "default_policy"
)

env = make_reach_env()
obs, info = env.reset()

# print(obs.dtype)
obs_batch = torch.from_numpy(obs.astype(np.float32)).unsqueeze(0)
model_outputs = rl_module.forward_inference({"obs": obs_batch})
print(model_outputs)
logits = model_outputs["action_dist_inputs"]
dist_class = rl_module.get_inference_action_dist_cls()
dist = dist_class.from_logits(logits)
action_sample = dist.sample()
action = action_sample.squeeze(0).detach().numpy().astype(np.float32)
print(action_sample)
print(action)
obs, reward, terminated, truncated, info = env.step(action)

print(f"obs:{obs}")
print(f"reward:{reward}")
print(f"terminated:{terminated}")
print(f"truncated:{truncated}")
print(f"info:{info}")


In [ ]:
import datetime
from ray import train, tune, air
from ray.rllib.algorithms.sac import SACConfig
from common.envUtils import *

TASK="Reach_"
experiment_name = TASK + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
LOGDIR=f"/home/ey/rl/src/rlreach2/rlreach/ray/db/ray_results/{experiment_name}"

config = (
    SACConfig()
    .environment(
        env=ReachEnvSimpleGym,
        env_config={"log_dir": LOGDIR},        
    )
    .training(
        initial_alpha=0.2,
        actor_lr=1e-4,
        critic_lr=1e-4,
        alpha_lr=1e-4,
        target_entropy="auto",
        n_step=1,
        tau=0.005,
        train_batch_size=128,
        target_network_update_freq=1,
        replay_buffer_config={
            "type": "EpisodeReplayBuffer",
            "capacity": 1000000,
            "learning_starts": 1000,
            # HER 专用参数
            "replay_mode": "independent",
            "replay_sequence_length": 1,
            "replay_burn_in": 0,
            "replay_zero_init_states": False,
            "storage_unit": "episodes",
            # 关键：HER wrapper 配置
            "wrap_buffer": True,
            "wrapped_buffer": {
                "type": "HindsightExperienceReplayBuffer",
                "replay_mode": "independent",
                "her_strategy": "future",      # 可选: future, final, episode
                "replay_k": 4,                 # 每个 transition 生成多少个 HER 样本
                "goal_fn": None,               # 你可以自定义 goal extraction function
            },
        },
        num_steps_sampled_before_learning_starts=1000,
        model={
            "fcnet_hiddens": [512, 512],
            "fcnet_activation": "relu",
            "post_fcnet_hiddens": [],
            "post_fcnet_activation": None,
            "post_fcnet_weights_initializer": "orthogonal_",
            "post_fcnet_weights_initializer_config": {"gain": 0.01},
        },
    )
    .resources(
        num_gpus=0.25,      # 或 0.25 视机器配置
        num_cpus_per_worker=1,
        num_learner_workers=1,
    )
    .framework("torch")
    .reporting(
        metrics_num_episodes_for_smoothing=5,
        min_sample_timesteps_per_iteration=1000,
    )
    .evaluation(
        evaluation_interval=1,
        evaluation_num_env_runners=1,
        evaluation_config={"seed": 42},
    )
    .env_runners(
        num_env_runners=6,             # 进程数量
        num_envs_per_env_runner=1,     # 环境数量
        # gym_env_vectorize_mode="ASYNC"
    )
)


tunner = tune.Tuner(
    trainable=config.algo_class,
    param_space=config,
    run_config=train.RunConfig(
        name="reach",
        storage_path=LOGDIR,
        log_to_file=True,
        checkpoint_config=air.CheckpointConfig(
            checkpoint_frequency=10,
            checkpoint_at_end=True,
        ),
        stop={"evaluation/env_runners/episode_return_mean": 10000.0}
    ),
)

results = tunner.fit()

2025-08-13 09:48:09,215	WARNING algorithm_config.py:5033 -- You are running SAC on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
2025-08-13 09:48:09,217	WARNING algorithm_config.py:5062 -- You configured a custom `model` config (probably through calling config.training(model=..), whereas your config uses the new API stack! In order to switch off the new API stack, set in your config: `config.api_stack(enable_rl_module_and_learner=False, enable_env_runner_and_connector_v2=False)`. If you DO want to use the new API stack, configure your model, instead, through: `config.rl_module(model_config={..})`.
2025-08-13 09:48:09,217	WARNING sac.py:487 -- You are running SAC on the new API stack! This is the new default

(SingleAgentEnvRunner pid=310808) [chatbus_3] 共享内存不存在，创建成功
(SingleAgentEnvRunner pid=310811) [chatbus_3] 共享内存已存在，连接成功


(SingleAgentEnvRunner pid=310810) Exception raised in creation task: The actor died because of an error raised in its creation task, ray::SingleAgentEnvRunner.__init__() (pid=310810, ip=192.168.202.162, actor_id=6251ab3e8e9da41fb9f4973201000000, repr=<ray.rllib.env.single_agent_env_runner.SingleAgentEnvRunner object at 0x7dec5e7242e0>)
(SingleAgentEnvRunner pid=310810)   File "/home/ey/anaconda3/envs/rlreach310/lib/python3.10/multiprocessing/shared_memory.py", line 104, in __init__
(SingleAgentEnvRunner pid=310810)     self._fd = _posixshmem.shm_open(
(SingleAgentEnvRunner pid=310810) FileNotFoundError: [Errno 2] No such file or directory: '/chatbus_3'
(SingleAgentEnvRunner pid=310810) 
(SingleAgentEnvRunner pid=310810) During handling of the above exception, another exception occurred:
(SingleAgentEnvRunner pid=310810) 
(SingleAgentEnvRunner pid=310810) ray::SingleAgentEnvRunner.__init__() (pid=310810, ip=192.168.202.162, actor_id=6251ab3e8e9da41fb9f4973201000000, repr=<ray.rllib.env.

(SingleAgentEnvRunner pid=311140) [chatbus_5] 共享内存不存在，创建成功 [repeated 2x across cluster]
(SingleAgentEnvRunner pid=310809) [chatbus_3] 共享内存已存在，连接成功


(pid=311210) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=311210) E0000 00:00:1755049708.914570  311210 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=311210) E0000 00:00:1755049708.918280  311210 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=311210) W0000 00:00:1755049708.927947  311210 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once. [repeated 4x across cluster]
(_WrappedExecutable pid=311210) [robosuite WARNING] No private macro file found! (macros.py:57)
(_WrappedExecutable pid=311210) [robosuite WARNING] It is recommended to use a private macro file (macros.py:58)
(_WrappedExecutable pid=311210) [robosuite WARNING] To setup, run: python /home/ey/rl/src/robos